In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from sklearn.utils import shuffle
import cv2  

from datacube import Datacube
import matplotlib.pyplot as plt

sys.path.append("../scripts")
from dea_plotting import map_shapefile
from dea_spatialtools import xr_rasterize

import tensorflow as tf
import os
import random

from skimage.io import imread, imshow
from skimage.transform import resize

dc = Datacube(app="Getting started")

IMG_WIDTH = 512
IMG_HEIGHT = 512

linescan_datasets = dc.find_datasets(product='linescan')
linescan_datasets = sorted(linescan_datasets, key = lambda ds: (ds.center_time, ds.id))
def clean_name(name):
    if name is None:
        res = None
    else:
        if name.upper()[-4::] == ".JPG":
            res = name.upper()[:-4].replace(' ','_')
        else:
            res = name.upper().replace(' ','_')
    return res

vector_file = 'resources/fire_boundaries.shp'
gdf = gpd.read_file(vector_file)

gdf['SourceNameClean'] = gdf.apply(lambda row: clean_name(row.SourceName), axis=1)
gdf.dtUTC = gdf.apply(lambda row: datetime.strptime(row.dtUTC, '%Y-%m-%d %H:%M:%S'), axis=1)
gdf.dtLocal = gdf.apply(lambda row: datetime.strptime(row.dtLocal, '%Y-%m-%d %H:%M:%S'), axis=1)

rejected_linescans = ['MACALISTER91_615_P1_201903051438_MGA94_55']
train_size = 0

for i in range(len(linescan_datasets)):
    sample = linescan_datasets[i]
    fname = sample.metadata_doc['label']
    
    if sum(gdf.SourceNameClean == fname) != 0 and fname not in rejected_linescans:
        train_size += 1 
        
train_size = 8 * train_size

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
X_test = np.load('X_test_224.npy')
data = np.load('train_224.npz')
X_train = data['a']
Y_train = data['b']

In [3]:
for i in range(X_test.shape[0]):   
    x_min = np.min(X_test[i])
    x_max = np.max(X_test[i])
    
    f = lambda x: (x - x_min) / (x_max - x_min)
    X_test[i] = f(X_test[i])

In [4]:
for i in range(X_train.shape[0]):   
    x_min = np.min(X_train[i])
    x_max = np.max(X_train[i])
    
    f = lambda x: (x - x_min) / (x_max - x_min)
    X_train[i] = f(X_train[i])

/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [5]:
for i in range(X_test.shape[0]):   
    x_min = np.min(X_test[i])
    x_max = np.max(X_test[i])
    print(x_min, x_max)
    
for i in range(X_train.shape[0]):   
    x_min = np.min(X_train[i])
    x_max = np.max(X_train[i])
    print(x_min, x_max)

np.savez_compressed('train_224_norm', a=X_train, b=Y_train)
np.save('X_test_224_norm.py', X_test)

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [ ]:
for i in range(5):
    lowerBound=np.array([80,80,80])
    upperBound=np.array([255,255,255])

    rgb_mask = cv2.inRange(X_test[i], lowerBound, upperBound)
    
    print(rgb_mask.dtype)
    print(rgb_mask.shape)

    f = plt.figure(figsize=(16, 16))
    f.add_subplot(1, 4, 1)
    plt.imshow(rgb_mask)
    
    #establish dialation and contraction parameters
    kernelOpen=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)) # try it yourself!
    kernelClose=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(32,32))

    #denoise the pixels
    maskOpen=cv2.morphologyEx(rgb_mask,cv2.MORPH_OPEN,kernelOpen)

    #plot the denoised pixels
    f.add_subplot(1, 4, 2)
    plt.imshow(maskOpen)
    
    #join the pxels together
    maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

    #plot
    f.add_subplot(1, 4, 3)
    plt.imshow(maskClose)
    
    kernel = np.ones((6,6),np.uint8)
    erosion = cv2.erode(maskClose,kernel,iterations = 1)
    
    f.add_subplot(1, 4, 4)
    plt.imshow(erosion)
    plt.show(block=True)